In [1]:
from Preprocessing import ImagePreprocessing
from Postprocessing import ImagePostprocessing
import numpy as np
import matplotlib.pyplot as plt
from PatchFeatureExtractor import PatchFeatureExtractor
pfe = PatchFeatureExtractor()

In [2]:
from ImageLoader import ImageLoader
import os

image_dir = "images/"
image_paths = [os.path.join(image_dir, "pictures", fname) for fname in os.listdir(os.path.join(image_dir, "pictures"))]
labels_paths = [
    os.path.join(image_dir, "manual", os.path.basename(p).replace(".JPG", ".tif").replace(".jpg", ".tif"))
    for p in image_paths
]
masks_paths = [
    os.path.join(image_dir, "mask", os.path.basename(p).replace(".JPG", "_mask.tif").replace(".jpg", "_mask.tif"))
    for p in image_paths
]

images = ImageLoader.load_images(image_paths)
labels = ImageLoader.load_images(labels_paths)
masks = ImageLoader.load_images(masks_paths)

images = np.stack(images)
labels = np.stack(labels)
masks = np.stack(masks)

In [3]:
print(images.shape, labels.shape, masks.shape)

(45, 2336, 3504, 3) (45, 2336, 3504) (45, 2336, 3504, 3)


In [4]:
from scipy.ndimage import convolve

class_patches_per_img = 1
patch_size = 5

patches = []
features = []
patch_labels = []

for image, label, mask in zip([images[0]], [labels[0]], [masks[0]]):
    positive_idx = []
    negative_idx = []
    half_patch = patch_size // 2

    # Convert mask to single channel if needed
    if mask.ndim == 3 and mask.shape[2] > 1:
        mask_gray = (mask[..., 0] > 0).astype(np.uint8)
    else:
        mask_gray = (mask > 0).astype(np.uint8)

    kernel = np.ones((patch_size, patch_size), dtype=np.uint8)
    valid_mask = convolve(mask_gray, kernel, mode='constant', cval=0)
    valid_mask = valid_mask == (patch_size * patch_size)

    for y in range(half_patch, label.shape[0] - half_patch):
        for x in range(half_patch, label.shape[1] - half_patch):
            if valid_mask[y, x]:
                if label[y, x] == 1:
                    positive_idx.append((y, x))
                elif label[y, x] == 0:
                    negative_idx.append((y, x))
    np.random.shuffle(positive_idx)
    np.random.shuffle(negative_idx)

    num_pos = min(len(positive_idx), class_patches_per_img)
    num_neg = min(len(negative_idx), class_patches_per_img)

    selected_pos = positive_idx[:num_pos]
    selected_neg = negative_idx[:num_neg]

    for y, x in selected_pos:
        patch = image[y - half_patch:y + half_patch + 1, x - half_patch:x + half_patch + 1]
        patches.append(patch)
        features.append(pfe.extract_features(patch))
        patch_labels.append(1)

    for y, x in selected_neg:
        patch = image[y - half_patch:y + half_patch + 1, x - half_patch:x + half_patch + 1]
        patches.append(patch)
        features.append(pfe.extract_features(patch))
        patch_labels.append(0)


In [5]:
print(len(patches), len(features))

1 1


In [6]:
# ~1m 18s per image 5x5
# ~ per image 27x27
patches, labels = pfe.extract_patches([images[0]], [labels[0]], [masks[0]], 27)

KeyboardInterrupt: 

In [ ]:
print(np.count(patches))
zero_indices = np.where(labels == 0)[0]
positive = np.count_nonzero(labels)
drop_count = len(labels) - 2 * positive
print(drop_count)

5810809


In [ ]:
# Randomly select indices to drop
drop_indices = np.random.choice(zero_indices, size=drop_count, replace=False)

# Create mask to keep the rest
mask = np.ones(len(labels), dtype=bool)
mask[drop_indices] = False

patches = patches[mask]
labels = labels[mask]

In [ ]:
print(np.count_nonzero(labels), len(labels))
print(patches.shape)

541198 1082396
(1082396, 5, 5, 3)


In [ ]:
# ~1m 17s
features = pfe.extract_features(patches, labels)

In [ ]:
print(features[0:5])

[[ 9.21600000e-01  7.20000000e-01  6.40000000e-01  3.03092013e+03
  -1.30798680e+01  3.03092013e+03  2.64106581e-03  1.29902103e-10
   2.99455178e-13  2.23102601e-13 -5.76664030e-26 -2.54280305e-18
   0.00000000e+00]
 [ 1.70560000e+00  1.19040000e+00  1.04640000e+00  3.05696799e+03
  -2.99634226e+01  3.06195820e+03  2.61050683e-03  6.58177664e-10
   2.45892227e-14  2.44440869e-13  1.57346464e-26  6.14780360e-18
   1.05623366e-26]
 [ 1.20000000e+00  1.20000000e+00  1.20000000e+00  2.90337931e+03
  -8.60689655e+00  2.90675103e+03  2.76343893e-03  6.96036108e-11
   6.05849867e-13  1.82631560e-12 -9.68115433e-25  1.07319542e-17
  -1.65931145e-24]
 [ 8.57600000e-01  8.57600000e-01  8.57600000e-01  2.88869417e+03
  -2.68543689e+01  2.88793065e+03  2.77807292e-03  6.67293065e-10
   1.52509710e-13  8.80508753e-13 -1.01602571e-25  1.60983529e-17
  -3.06248878e-25]
 [ 7.20000000e-01  1.03040000e+00  1.04000000e+00  2.87793031e+03
  -2.99233449e+01  2.86691568e+03  2.78980976e-03  8.73252952e-10
